In [1]:
import pandas as pd
import numpy as np

from quartile import *
from random import sample



b'Skipping line 32712: expected 20 fields, saw 21\nSkipping line 32713: expected 20 fields, saw 21\n'
b'Skipping line 20760: expected 20 fields, saw 21\nSkipping line 22914: expected 20 fields, saw 21\n'
b'Skipping line 19393: expected 20 fields, saw 21\nSkipping line 19394: expected 20 fields, saw 21\n'
b'Skipping line 33757: expected 20 fields, saw 21\n'
b'Skipping line 19053: expected 20 fields, saw 21\nSkipping line 20953: expected 20 fields, saw 21\n'


## NOTE: Can directly jump to PART 3!

### PART 1: Adding Sbj. Area to retr_onetwo

### After Bruno meeting:

Need to create 4 pivot tables for finalizing the sample count with Prof. Bruno

Will tidy up the nb, as well as creating the pivots below:

In [2]:
# Retrieving both important dfs:
 
%store -r retr_onetwo
retr_onetwo = retr_onetwo

%store -r date_df_ok
date_df_ok = date_df_ok


In [8]:
# Creting the quartile.py variables:

# Get q_df(s):
q_df = quar_df_creator(1).reset_index(drop=True)

# Create q_jrnl & q_pivor using q_df & article count target:
q_jrnl, sbj_pivot_full, sbj_pivot_sample = retr_ready(q_df,100000)

b'Skipping line 32712: expected 20 fields, saw 21\nSkipping line 32713: expected 20 fields, saw 21\n'
b'Skipping line 20760: expected 20 fields, saw 21\nSkipping line 22914: expected 20 fields, saw 21\n'
b'Skipping line 19393: expected 20 fields, saw 21\nSkipping line 19394: expected 20 fields, saw 21\n'
b'Skipping line 33757: expected 20 fields, saw 21\n'
b'Skipping line 19053: expected 20 fields, saw 21\nSkipping line 20953: expected 20 fields, saw 21\n'


In [11]:
# Normally, from q_jrnl we .groupby() to get Total Docs summed up:
q_ready = q_jrnl.groupby([q_jrnl["Issn"].map(tuple),q_jrnl["Year"]])["Total_Docs"].sum().reset_index()

# However, one missing feature from retr_onetwo is the "Sbj_Area"s of the journals. 
# As this feature will be used in this nb & as well as in the future analysis, this feat. will be added below:

q_sbj_areas = q_jrnl.groupby([q_jrnl["Issn"].map(tuple),q_jrnl["Year"]])["Sbj_Area"].apply(tuple).reset_index()
q_sbj_areas["Issn"] = q_sbj_areas.Issn.map(lambda x: [iss for iss in x])

In [35]:
# As the main df we have been using is retr_onetwo, it would be more beneficial to add this info directly to it:


#  Checking if retr_onetwo and the newly created q_sbj_areas have the same index before merging:
# (after dtype matching)

print(retr_onetwo["Issn"].equals(q_sbj_areas.Issn))
print(retr_onetwo["Year"].equals(q_sbj_areas.Year))

# Adding the column to retr_onetwo
retr_onetwo["Sbj_Area"] = q_sbj_areas.Sbj_Area


True
True


The new df will be re-ordered and will be stored seperately. This new df will be the main df for Q1 journals from now on:

In [37]:
retr_q1_ok = retr_onetwo[["Issn","Year","Sbj_Area","CR_Results","SH_Results","Total_Docs"]]

In [177]:
%store retr_q1_ok

Stored 'retr_q1_ok' (DataFrame)


### PART 1.5: Creating Bruno pivots

As retr_q1_ok now has Sbj. Area & Total Docs info together, we can use explade() seperately from the pipline to create any pivot table required.

In [51]:
pladed_q1 = explade(retr_q1_ok) 

In [ ]:
# Bruno pivots:

# 1- FULL - YEAR vs ARTICLE Count
bruno1 = pd.pivot_table(data=pladed_q1, columns="Year", index="Sbj_Area", values="Total_Docs", aggfunc=sum)

# 2- DATE_OK - YEAR vs. ARTICLE Count
bruno2 = pd.pivot_table(data=pladed_q1[(pladed_q1.CR_Results == True) & (pladed_q1.SH_Results == True)], columns="Year", index="Sbj_Area", values="Total_Docs", aggfunc=sum)

# 3- FULL - YEAR vs JOURNAL Count
bruno3 = pd.pivot_table(data=pladed_q1, columns="Year", index="Sbj_Area", values="Total_Docs", aggfunc="count")

# 4- DATE_OK - YEAR vs. JOURNAL Count
bruno4 = pd.pivot_table(data=pladed_q1[(pladed_q1.CR_Results == True) & (pladed_q1.SH_Results == True)], columns="Year", index="Sbj_Area", values="Total_Docs", aggfunc="count")


In [78]:
with pd.ExcelWriter("pivot_tables.xlsx") as writer:

    bruno1.to_excel(writer, sheet_name="FULL - YEAR vs ARTICLE")  

    bruno2.to_excel(writer, sheet_name="DATE_OK - YEAR vs. ARTICLE")

    bruno3.to_excel(writer,sheet_name="FULL - YEAR vs JOURNAL")

    bruno4.to_excel(writer,sheet_name="DATE_OK - YEAR vs. JOURNAL")

### PART 2: Correcting Sbj. Area 

During evaluation, it is seen that the multimode simplification during q_df process is actually unnecessary. A journal can be represented with all the Sbj. Areas in its highest quartile. The cells below will test this using quartile_v2.py 

In [79]:
from quartile_v2 import *

In [80]:
q_test = quar_df_creator(1).reset_index(drop=True)


b'Skipping line 32712: expected 20 fields, saw 21\nSkipping line 32713: expected 20 fields, saw 21\n'
b'Skipping line 20760: expected 20 fields, saw 21\nSkipping line 22914: expected 20 fields, saw 21\n'
b'Skipping line 19393: expected 20 fields, saw 21\nSkipping line 19394: expected 20 fields, saw 21\n'
b'Skipping line 33757: expected 20 fields, saw 21\n'
b'Skipping line 19053: expected 20 fields, saw 21\nSkipping line 20953: expected 20 fields, saw 21\n'


In [245]:
q_test_ready = retr_ready_v2(q_test)

In [154]:
test_sbj_areas = q_test_ready.groupby([q_test_ready["Issn"].map(tuple),q_test_ready["Year"]])["Sbj_Area"].apply(tuple).reset_index()
#test_sbj_areas["Issn"] = test_sbj_areas.Issn.map(lambda x: [iss for iss in x])

The easiest way to combine these 2 dfs would be to use ISSN & Year cols. as index as they together uniquely define a journal. If this method works, would be a huge ease of use for future jnrl df related works as well.

In [214]:
test_sbj_areas.set_index(["Issn","Year"], inplace=True)

# Realised that the dtypes of Year columns don't match, so converting to int64 before indexing:

test_sbj_areas.reset_index(inplace=True)
test_sbj_areas["Year"] = test_sbj_areas["Year"].astype(np.int64)
test_sbj_areas.set_index(["Issn","Year"], inplace=True)

In [196]:
# Set index for retr_q1_ok 
retr_q1_ok["Issn"] = retr_q1_ok.Issn.map(tuple)
retr_q1_ok.set_index(["Issn","Year"], inplace=True)

In [224]:
retr_q1_ok = pd.merge(left=retr_q1_ok, right=test_sbj_areas, how="left", right_index=True, left_index=True, suffixes=("_old", "_new"))

In [227]:
retr_q1_ok.dropna(inplace=True)

Sbj_Area_old  \
Issn                 Year                                                      
(00011541, 15475905) 2010  (Biochemistry, Genetics and Molecular Biology,...   
                     2011      (Chemical Engineering, Environmental Science)   
                     2012      (Chemical Engineering, Environmental Science)   
                     2013  (Biochemistry, Genetics and Molecular Biology,...   
                     2014  (Biochemistry, Genetics and Molecular Biology,...   
...                                                                      ...   
(8756758X, 14602695) 2017                                     (Engineering,)   
                     2018                                     (Engineering,)   
                     2019                                     (Engineering,)   
                     2020                                     (Engineering,)   
(8756971X, 19436270) 2010                                        (Medicine,)   

                          CR_Results  SH_Results  Total_Docs  \
Issn                 Year                                      
(00011541, 15475905) 2010       True        True         294   
                     2011       True       False         315   
                     2012       True       False         347   
                     2013       True        True         422   
                     2014       True        True         359   
...                              ...         ...         ...   
(8756758X, 14602695) 2017       True        True         170   
                     2018       True        True         199   
                     2019       True        True         210   
                     2020       True        True         221   
(8756971X, 19436270) 2010       True       False          69   

                                                                Sbj_Area_new  
Issn                 Year                                                     
(00011541, 15475905) 2010  (Biochemistry, Genetics and Molecular Biology,...  
                     2011      (Chemical Engineering, Environmental Science)  
                     2012      (Chemical Engineering, Environmental Science)  
                     2013  (Biochemistry, Genetics and Molecular Biology,...  
                     2014  (Biochemistry, Genetics and Molecular Biology,...  
...                                                                      ...  
(8756758X, 14602695) 2017                   (Materials Science, Engineering)  
                     2018                   (Materials Science, Engineering)  
                     2019                   (Materials Science, Engineering)  
                     2020                   (Materials Science, Engineering)  
(8756971X, 19436270) 2010   (Agricultural and Biological Sciences, Medicine)  

[74400 rows x 5 columns]

In [250]:
%store retr_q1_ok

Stored 'retr_q1_ok' (DataFrame)


It is seen that the index merging works perfectly and is very easy to use! 

However, we could have directly merged the Counter version of Sbj. Areas to retr_q1_ok so that we explade only that df, and create pivots. Now that the way of working is decided, will save new dfs, and reset the nb to finalize all work.


### PART 3: Revenge of the Counter

In [1]:
import pandas as pd
import numpy as np

from quartile_v2 import *

b'Skipping line 32712: expected 20 fields, saw 21\nSkipping line 32713: expected 20 fields, saw 21\n'
b'Skipping line 20760: expected 20 fields, saw 21\nSkipping line 22914: expected 20 fields, saw 21\n'
b'Skipping line 19393: expected 20 fields, saw 21\nSkipping line 19394: expected 20 fields, saw 21\n'
b'Skipping line 33757: expected 20 fields, saw 21\n'
b'Skipping line 19053: expected 20 fields, saw 21\nSkipping line 20953: expected 20 fields, saw 21\n'


In [5]:
%store -r retr_onetwo
retr_onetwo = retr_onetwo

# %store -r date_df_ok
# date_df_ok = date_df_ok


In [13]:
# Creting the quartile.py variables:

# Get q_df(s):
q_part3 = quar_df_creator_v2(1).reset_index(drop=True)


b'Skipping line 32712: expected 20 fields, saw 21\nSkipping line 32713: expected 20 fields, saw 21\n'
b'Skipping line 20760: expected 20 fields, saw 21\nSkipping line 22914: expected 20 fields, saw 21\n'
b'Skipping line 19393: expected 20 fields, saw 21\nSkipping line 19394: expected 20 fields, saw 21\n'
b'Skipping line 33757: expected 20 fields, saw 21\n'
b'Skipping line 19053: expected 20 fields, saw 21\nSkipping line 20953: expected 20 fields, saw 21\n'


In [9]:
q_part3 = q_part3[["Issn", "Year", "Sbj_Area"]]

In [23]:
# Convert both Issn columns to tuples:

q_part3["Issn"] = q_part3["Issn"].map(tuple)
retr_onetwo["Issn"] = retr_onetwo.Issn.map(tuple)

# Convert YEAR column to type int:
q_part3["Year"] = q_part3.Year.astype(int)

# Set Issn & Year cols. as index for merging:
q_part3.set_index(["Issn", "Year"], inplace= True)
retr_onetwo.set_index(["Issn", "Year"], inplace=True)

# Merging both to create retr_q1 df:
retr_q1 = pd.merge(retr_onetwo, q_part3, left_index=True, right_index=True, suffixes=("_old", "_new"), how="left") 

The resulting df retr_q1 has 2 rows extra, as q_part3 df has 2 duplicate columns. These are idecntified and dropped below:

In [47]:
q_part3.reset_index()[q_part3.reset_index().Issn.map(lambda x: "13474065" in x)]

,Issn,Year,Sbj_Area
2401,"(00214922, 13474065)",2010,"{'Engineering': 1, 'Physics and Astronomy': 1}"
5097,"(13474065, 00214922)",2010,{'Engineering': 1}
5138,"(13474065, 00214922)",2010,{'Engineering': 1}
11208,"(13474065, 00214922)",2011,{'Engineering': 1}
11296,"(13474065, 00214922)",2011,{'Engineering': 1}
17891,"(13474065, 00214922)",2012,{'Engineering': 1}
24518,"(13474065, 00214922)",2013,{'Engineering': 1}
45311,"(13474065, 00214922)",2016,{'Engineering': 1}
52296,"(13474065, 00214922)",2017,{'Engineering': 1}
59506,"(13474065, 00214922)",2018,{'Engineering': 1}


In [54]:
q_part3.reset_index()

,Issn,Year,Sbj_Area
0,"(00346861, 15390756)",2010,{'Physics and Astronomy': 1}
1,"(15453278, 07320582)",2010,"{'Immunology and Microbiology': 1, 'Medicine': 1}"
2,"(15454509, 00664154)",2010,"{'Biochemistry, Genetics and Molecular Biology..."
3,"(10614036, 15461718)",2010,"{'Biochemistry, Genetics and Molecular Biology..."
4,"(15424863, 00079235)",2010,{'Medicine': 2}
...,...,...,...
75525,"(07303238, 15489590)",2020,{'Arts and Humanities': 1}
75526,"(01234412,)",2020,{'Arts and Humanities': 1}
75527,"(16551532, 20120788)",2020,{'Arts and Humanities': 1}
75528,"(05836255,)",2020,{'Arts and Humanities': 1}


In [53]:
q_part3.reset_index().drop_duplicates(subset=("Issn", "Year"))

,Issn,Year,Sbj_Area
0,"(00346861, 15390756)",2010,{'Physics and Astronomy': 1}
1,"(15453278, 07320582)",2010,"{'Immunology and Microbiology': 1, 'Medicine': 1}"
2,"(15454509, 00664154)",2010,"{'Biochemistry, Genetics and Molecular Biology..."
3,"(10614036, 15461718)",2010,"{'Biochemistry, Genetics and Molecular Biology..."
4,"(15424863, 00079235)",2010,{'Medicine': 2}
...,...,...,...
75525,"(07303238, 15489590)",2020,{'Arts and Humanities': 1}
75526,"(01234412,)",2020,{'Arts and Humanities': 1}
75527,"(16551532, 20120788)",2020,{'Arts and Humanities': 1}
75528,"(05836255,)",2020,{'Arts and Humanities': 1}


In [77]:
# rop duplicates of retr_q1 to get the same number of rows as retr_onetwo:
retr_q1 = retr_q1.reset_index().drop_duplicates(subset=("Issn", "Year"))

# Re-ordering columns for ease of use:
retr_q1 = retr_q1[["Issn","Year","Sbj_Area","CR_Results","SH_Results","CR_Count","Total_Docs"]]

# Setting index back after drop duplicate process:
retr_q1.set_index(["Issn","Year"],inplace=True) 

# Store the resulting df:
%store retr_q1

In [5]:
# Need to reset index to use explade function:
retr_q1.reset_index(inplace=True)

# Creating expladed_df to create pivot tables: 
expladed_q1 = explade_v2(retr_q1)

In [6]:
# Bruno pivots (v2):

# 1- FULL - YEAR vs ARTICLE Count
bruno1 = pd.pivot_table(data=expladed_q1, columns="Year", index="Sbj_Area", values="Total_Docs", aggfunc=sum)

# 2- DATE_OK - YEAR vs. ARTICLE Count
bruno2 = pd.pivot_table(data=expladed_q1[(expladed_q1.CR_Results == True) & (expladed_q1.SH_Results == True)], columns="Year", index="Sbj_Area", values="Total_Docs", aggfunc=sum)

# 3- FULL - YEAR vs JOURNAL Count
bruno3 = pd.pivot_table(data=expladed_q1, columns="Year", index="Sbj_Area", values="Total_Docs", aggfunc="count")

# 4- DATE_OK - YEAR vs. JOURNAL Count
bruno4 = pd.pivot_table(data=expladed_q1[(expladed_q1.CR_Results == True) & (expladed_q1.SH_Results == True)], columns="Year", index="Sbj_Area", values="Total_Docs", aggfunc="count")


In [12]:
with pd.ExcelWriter("pivot_tables_v2.xlsx") as writer:

    bruno1.to_excel(writer, sheet_name="FULL - ARTICLE")  

    bruno2.to_excel(writer, sheet_name="DATE_OK - ARTICLE")

    bruno3.to_excel(writer,sheet_name="FULL - JOURNAL")

    bruno4.to_excel(writer,sheet_name="DATE_OK - JOURNAL")

### FINAL NOTE:

Depending on Bruno's answer, we are ready to create sample counts & finalize q1 process both for journals & articles.

In the meantime, we can use this time to do the same for q2 journals.

Up to this point, there are 2 variables that are complete & ready to be used. Also, there are 3 date_df_all dfs from 5b SumCetCet run.

* Journal df -> retr_q1
* Article df -> date_df_ok

TTD:
* Check run results from 5b, update date_df_ok if necessary
* Create retr_q2 complete with CR_Results, SH_Results columns
* Check ScopusAPI for Affiliation

---
---


### AFTER BRUNO MEETING:

The meeting was successful & now there is a new way forward with the project.

* Waiting on Bruno's reply to pivots; afterwards we can start the Q1 jrnl_df final sampling process. Almost everything is ready:
    * Jrnl info is ready & with the new & improved retr_q1 dataset
    * Artl count is also available

_Meanwhile_:

1- need to test & implement the new Affiliation extraction:
* __Scopus API Author Search__
    * Testing the API & requirements
    * Creating an author_df & integrating to retr_q1
    * get affiliations for both

2- Create & prepare retr_q2 dataset:

* Simpler & more time consuming:
* Need to repeat the jrnl pipeline for Q2 jrnls per Bruno's request
    * Scimago -> retr_ready
    * CR_Check -> can simply use CR_Count instead: if 0 -> False, else True
    * SH_Check -> if CR_check True, get 2 DOIs, modify & run second_run pipeline & fill the column


Therefore, will start a new notebook for Scopus API Author testing, anohe rone for retr_q2 & will create a pipeline for jrnl processes and finally once Bruno replies another one for creating final q1_artcl df!

So, this nb is completed with 2 crucial variables:

* date_df_ok -> Q1 jrnls metadata retrieved articles
    * BUT there is actually a newer one from 5b but need to concat SumCetCet run results!
* retr_q1 -> Q1 jrnl_df ready to be used for sampling & other analysis
